In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models     import Sequential
from keras.layers     import Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
#Dostępność karty graficznej 
tf.test.is_gpu_available()

True

In [27]:
#Importuje wcześniej przygotowane dane z różnym stopniem preprocessingu
#bez zmiennych kategorycznych 
house_data=pd.read_csv("data_to_import.csv")
#z dodatkową zmienną uzyskaną z clusteryzacji
house_data_full=pd.read_csv("data_to_import_full.csv")
#ze zmiennymi kategorycznymi
house_data_factor=pd.read_csv("data_to_import_factor.csv")

In [10]:
house_data_full.head()

,price,bedrooms,bathrooms,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,when_sold,district
0,221900.0,3,1.00,5650,1.0,0,0,3,7,1180,0,1955,0,1340,5650,1634,1
1,538000.0,3,2.25,7242,2.0,0,0,3,7,2170,400,1951,1991,1690,7639,1577,1
2,180000.0,2,1.00,10000,1.0,0,0,3,6,770,0,1933,0,2720,8062,1499,2
3,604000.0,4,3.00,5000,1.0,0,0,5,7,1050,910,1965,0,1360,5000,1577,1
4,510000.0,3,2.00,8080,1.0,0,0,3,8,1680,0,1987,0,1800,7503,1506,2


In [11]:
#Preprocesing, na razie bez standaryzacji
house_data[['view']]=house_data[["view"]].astype("category")
house_data[['waterfront']]=house_data[["waterfront"]].astype("category")
house_data_full[['view']]=house_data[["view"]].astype("category")
house_data_full[['waterfront']]=house_data[["waterfront"]].astype("category")
house_data_factor[['view']]=house_data_factor[["view"]].astype("category")
house_data_factor[['waterfront']]=house_data_factor[["waterfront"]].astype("category")
house_data_full[['district']]=house_data_full[["district"]].astype("category")
house_data_factor[['district']]=house_data_factor[["district"]].astype("category")
house_data_factor[['is_basement']]=house_data_factor[["is_basement"]].astype("category")
house_data_factor[["is_renovated"]]=house_data_factor[["is_renovated"]].astype("category")

In [28]:
house_data_factor=pd.get_dummies(house_data_factor, columns=["view", "waterfront", "district", "is_basement", "is_renovated"])
house_data_full=pd.get_dummies(house_data_full, columns=["view", "waterfront", "district"])
house_data=pd.get_dummies(house_data, columns=["view", "waterfront"])

In [29]:
house_data_values = house_data.values
house_data_factor_values = house_data_factor.values
house_data_full_values = house_data_full.values

In [30]:
X = house_data_values[:,1:]
y = house_data_values[:, 0]
X_factor = house_data_factor_values[:,1:]
y_factor = house_data_factor_values[:, 0]
X_full = house_data_full_values[:,1:]
y_full = house_data_full_values[:, 0]

In [31]:
#Podział na zbiór testowy i trenigowy, identyczny dla każdego zbioru danych
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1875)
X_train_fc, X_test_fc, y_train_fc, y_test_fc = train_test_split(X_factor, y_factor, test_size=0.4, random_state=1875)
X_train_fl, X_test_fl, y_train_fl, y_test_fl = train_test_split(X_full, y_full, test_size=0.4, random_state=1875)

In [24]:
#seria modeli, dla każdego zbioru danych z innym wejściem
model1=Sequential()
model1.add(Dense(150, input_shape=(X_train.shape[1],), activation='relu'))
model1.add(Dense(150, activation='relu'))
model1.add(Dense(150, activation='relu'))
model1.add(Dense(150, activation="relu"))
model1.add(Dense(1, activation="relu"))
model1.compile(loss='mse', optimizer='Adam')

model1_f=Sequential()
model1_f.add(Dense(150, input_shape=(X_train_fl.shape[1],), activation='relu'))
model1_f.add(Dense(150, activation='relu'))
model1_f.add(Dense(150, activation='relu'))
model1_f.add(Dense(150, activation="relu"))
model1_f.add(Dense(1, activation="relu"))
model1_f.compile(loss='mse', optimizer='Adam')

            

In [32]:
model1_fc=Sequential()
model1_fc.add(Dense(150, input_shape=(X_train_fc.shape[1],), activation='relu'))
model1_fc.add(Dense(150, activation='relu'))
model1_fc.add(Dense(150, activation='relu'))
model1_fc.add(Dense(150, activation="relu"))
model1_fc.add(Dense(1, activation="relu"))
model1_fc.compile(loss='mse', optimizer='Adam')

In [144]:
#funckcja do uczenia napisanych modeli
def wytrenuj_siec(model, x_train, x_test, train_y, test_y, epoki=10, rozmiar_batcha=None):
    model.fit(x_train, train_y, epochs=epoki, batch_size=rozmiar_batcha)
    result=model.evaluate(x_test, test_y)
    return np.sqrt(result)

In [25]:
result1=wytrenuj_siec(model1, X_train, X_test, y_train, y_test, 40)

Epoch 1/40
12967/12967 [==============================] - 2s 132us/step - loss: 137328185513.6269
Epoch 2/40
12967/12967 [==============================] - 1s 88us/step - loss: 68014093399.2615
Epoch 3/40
12967/12967 [==============================] - 1s 88us/step - loss: 66229398349.1336
Epoch 4/40
12967/12967 [==============================] - 1s 88us/step - loss: 63468884186.5881
Epoch 5/40
12967/12967 [==============================] - 1s 88us/step - loss: 60407713112.2289
Epoch 6/40
12967/12967 [==============================] - 1s 88us/step - loss: 59410706655.6422
Epoch 7/40
12967/12967 [==============================] - 1s 88us/step - loss: 57081561023.8766
Epoch 8/40
12967/12967 [==============================] - 1s 89us/step - loss: 56148360979.5647
Epoch 9/40
12967/12967 [==============================] - 1s 89us/step - loss: 55861008963.9534
Epoch 10/40
12967/12967 [==============================] - 1s 89us/step - loss: 56353527835.4815
Epoch 11/40
12967/12967 [============

In [26]:
result1_fl=wytrenuj_siec(model1_f, X_train_fl, X_test_fl, y_train_fl, y_test_fl, 40)
print(result1_fl, result1_fc, result1)

Epoch 1/40
12967/12967 [==============================] - 2s 151us/step - loss: 167121994113.6880
Epoch 2/40
12967/12967 [==============================] - 1s 112us/step - loss: 73630264960.5232
Epoch 3/40
12967/12967 [==============================] - 1s 93us/step - loss: 69243573891.5240
Epoch 4/40
12967/12967 [==============================] - 1s 91us/step - loss: 65760102125.6593
Epoch 5/40
12967/12967 [==============================] - 1s 91us/step - loss: 65286809985.8459
Epoch 6/40
12967/12967 [==============================] - 1s 90us/step - loss: 62283408314.5856
Epoch 7/40
12967/12967 [==============================] - 1s 97us/step - loss: 60572904989.9690
Epoch 8/40
12967/12967 [==============================] - 1s 96us/step - loss: 59954692885.2231
Epoch 9/40
12967/12967 [==============================] - 1s 93us/step - loss: 59370316195.8029
Epoch 10/40
12967/12967 [==============================] - 1s 91us/step - loss: 57186700005.6833
Epoch 11/40
12967/12967 [===========

In [33]:
result1_fc=wytrenuj_siec(model1_fc, X_train_fc, X_test_fc, y_train_fc, y_test_fc, 40)

Epoch 1/40
12967/12967 [==============================] - 2s 135us/step - loss: 103953491264.2221
Epoch 2/40
12967/12967 [==============================] - 1s 89us/step - loss: 60727092715.5863
Epoch 3/40
12967/12967 [==============================] - 1s 89us/step - loss: 58094303626.5326
Epoch 4/40
12967/12967 [==============================] - 1s 89us/step - loss: 57256150834.3629
Epoch 5/40
12967/12967 [==============================] - 1s 89us/step - loss: 56548852155.0200
Epoch 6/40
12967/12967 [==============================] - 1s 89us/step - loss: 56239400564.9936
Epoch 7/40
12967/12967 [==============================] - 1s 91us/step - loss: 56239644025.7120
Epoch 8/40
12967/12967 [==============================] - 1s 89us/step - loss: 55898998946.9934
Epoch 9/40
12967/12967 [==============================] - 1s 89us/step - loss: 55716250859.7245
Epoch 10/40
12967/12967 [==============================] - 1s 93us/step - loss: 55220069512.2227
Epoch 11/40
12967/12967 [============

In [34]:
print(result1_fc)

201714.3520360583


In [55]:
model2=Sequential()
model2.add(Dense(200, input_shape=(X_train.shape[1],), activation='relu'))
model2.add(Dense(200, activation='relu'))
model2.add(Dense(150, activation='relu'))
model2.add(Dense(150, activation="relu"))
model2.add(Dense(1, activation="relu"))
model2.compile(loss='mse', optimizer='Adam')

model2_f=Sequential()
model2_f.add(Dense(200, input_shape=(X_train_fl.shape[1],), activation='relu'))
model2_f.add(Dense(200, activation='relu'))
model2_f.add(Dense(150, activation='relu'))
model2_f.add(Dense(150, activation='relu'))
model2_f.add(Dense(1, activation="relu"))
model2_f.compile(loss='mse', optimizer='Adam')


In [56]:
result2=wytrenuj_siec(model2, X_train, X_test, y_train, y_test, 40)
result2_fl=wytrenuj_siec(model2_f, X_train_fl, X_test_fl, y_train_fl, y_test_fl, 40)
result2_fc=wytrenuj_siec(model2_fc, X_train_fc, X_test_fc, y_train_fc, y_test_fc, 40)

Epoch 1/40
12967/12967 [==============================] - 3s 239us/step - loss: 144811062939.4518
Epoch 2/40
12967/12967 [==============================] - 1s 106us/step - loss: 69802151171.2575
Epoch 3/40
12967/12967 [==============================] - 1s 113us/step - loss: 67488832876.4451
Epoch 4/40
12967/12967 [==============================] - 1s 114us/step - loss: 65485500682.2858
Epoch 5/40
12967/12967 [==============================] - 1s 102us/step - loss: 63927911625.6096
Epoch 6/40
12967/12967 [==============================] - 1s 99us/step - loss: 61451430164.5519
Epoch 7/40
12967/12967 [==============================] - 1s 105us/step - loss: 59153514932.5445
Epoch 8/40
12967/12967 [==============================] - 1s 112us/step - loss: 57664262582.7556
Epoch 9/40
12967/12967 [==============================] - 1s 112us/step - loss: 56947151393.5226
Epoch 10/40
12967/12967 [==============================] - 1s 102us/step - loss: 55704315247.4460
Epoch 11/40
12967/12967 [====

In [58]:
print(result2,result2_fl, result2_fc)

221262.7045407464 221583.1127349222 200236.52070276477


In [89]:
model2_fc=Sequential()
model2_fc.add(Dense(200, input_shape=(X_train_fc.shape[1],), activation='relu'))
model2_fc.add(Dense(200, activation='relu'))
model2_fc.add(Dense(200, activation='relu'))
model2_fc.add(Dense(200, activation="relu"))
model2_fc.add(Dense(1, activation="relu"))
model2_fc.compile(loss='mse', optimizer='Adam')

In [87]:
result2_fc=wytrenuj_siec(model2_fc, X_train_fc, X_test_fc, y_train_fc, y_test_fc, 50)

Epoch 1/10
12967/12967 [==============================] - 1s 114us/step - loss: 35525765015.2862
Epoch 2/10
12967/12967 [==============================] - 1s 115us/step - loss: 36690345919.0869
Epoch 3/10
12967/12967 [==============================] - 1s 111us/step - loss: 36881864350.8081
Epoch 4/10
12967/12967 [==============================] - 1s 112us/step - loss: 40230486046.4033
Epoch 5/10
12967/12967 [==============================] - 1s 112us/step - loss: 36831394744.6904
Epoch 6/10
12967/12967 [==============================] - 1s 112us/step - loss: 37343929851.2223
Epoch 7/10
12967/12967 [==============================] - 1s 113us/step - loss: 36459879988.0805
Epoch 8/10
12967/12967 [==============================] - 1s 112us/step - loss: 36701724868.3976
Epoch 9/10
12967/12967 [==============================] - 1s 113us/step - loss: 35900763100.7005
Epoch 10/10
8646/8646 [==============================] - 0s 45us/step


In [88]:
print(result2_fc)

201901.7855053158


In [68]:
#Standaryzujemy dane
scaler=StandardScaler()
scaler2=StandardScaler()
scaler3=StandardScaler()
names=house_data.columns
names_full=house_data_full.columns
names_factor=house_data_factor.columns

In [95]:
sc_house_data=scaler.fit_transform(house_data.iloc[:, 1:])
sc_house_data_factor=scaler2.fit_transform(house_data.iloc[:,1:])
sc_house_data_full=scaler3.fit_transform(house_data_full.iloc[:, 1:])

/home/arkadiusz/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/arkadiusz/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/arkadiusz/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/arkadiusz/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/arkadiusz/anaconda3/lib/python3.7/site-pac

In [96]:
X_sc = sc_house_data
y_sc = house_data_values[:, 0]
X_factor_sc = sc_house_data_factor
y_factor_sc = house_data_factor_values[:, 0]
X_full_sc = sc_house_data_full
y_full_sc = house_data_full_values[:, 0]

In [102]:
X_sc_train, X_sc_test, y_sc_train, y_sc_test = train_test_split(X_sc, y_sc, test_size=0.4, random_state=1875)
X_sc_train_fc, X_sc_test_fc, y_sc_train_fc, y_sc_test_fc = train_test_split(X_factor_sc, y_factor_sc, test_size=0.4, random_state=1875)
X_sc_train_fl, X_sc_test_fl, y_sc_train_fl, y_sc_test_fl = train_test_split(X_full_sc, y_full_sc, test_size=0.4, random_state=1875)

In [117]:
model1_sc=Sequential()
model1_sc.add(Dense(150, input_shape=(X_sc_train.shape[1],), activation='relu'))
model1_sc.add(Dense(150, activation='relu'))
model1_sc.add(Dense(150, activation='relu'))
model1_sc.add(Dense(150, activation="relu"))
model1_sc.add(Dense(1, activation="relu"))
model1_sc.compile(loss='mse', optimizer='Adam')
#Na razie winner
model1_sc_f=Sequential()
model1_sc_f.add(Dense(150, input_shape=(X_sc_train_fl.shape[1],), activation='relu'))
model1_sc_f.add(Dense(150, activation='relu'))
model1_sc_f.add(Dense(150, activation='relu'))
model1_sc_f.add(Dense(150, activation="relu"))
model1_sc_f.add(Dense(1, activation="relu"))
model1_sc_f.compile(loss='mse', optimizer='Adam')

model1_sc_fc=Sequential()
model1_sc_fc.add(Dense(150, input_shape=(X_sc_train_fc.shape[1],), activation='relu'))
model1_sc_fc.add(Dense(150, activation='relu'))
model1_sc_fc.add(Dense(150, activation='relu'))
model1_sc_fc.add(Dense(150, activation="relu"))
model1_sc_fc.add(Dense(1, activation="relu"))
model1_sc_fc.compile(loss='mse', optimizer='Adam')

In [118]:
result1_sc=wytrenuj_siec(model1_sc, X_sc_train, X_sc_test, y_sc_train, y_sc_test, 30)
result1_sc_fc=wytrenuj_siec(model1_sc_fc, X_sc_train_fc, X_sc_test_fc, y_sc_train_fc, y_sc_test_fc, 50)
result1_sc_fl=wytrenuj_siec(model1_sc_f, X_sc_train_fl, X_sc_test_fl, y_sc_train_fl, y_sc_test_fl, 50)

Epoch 1/30
12967/12967 [==============================] - 4s 325us/step - loss: 172099210853.3576
Epoch 2/30
12967/12967 [==============================] - 2s 135us/step - loss: 45593939709.3743
Epoch 3/30
12967/12967 [==============================] - 2s 126us/step - loss: 42364155646.7167
Epoch 4/30
12967/12967 [==============================] - 2s 127us/step - loss: 41100834258.6319
Epoch 5/30
12967/12967 [==============================] - 2s 126us/step - loss: 40269565650.3358
Epoch 6/30
12967/12967 [==============================] - 2s 125us/step - loss: 39702944878.3207
Epoch 7/30
12967/12967 [==============================] - 2s 125us/step - loss: 39189741454.4416
Epoch 8/30
12967/12967 [==============================] - 2s 125us/step - loss: 38652574916.8714
Epoch 9/30
12967/12967 [==============================] - 2s 126us/step - loss: 38078610780.6512
Epoch 10/30
12967/12967 [==============================] - 2s 125us/step - loss: 38084094625.6115
Epoch 11/30
12967/12967 [===

In [119]:
print(result1_sc, result1_sc_fl, result1_sc_fc)
#Jest widoczna znaczna poprawa

187080.0264980009 160365.90081121764 182070.4699030122


In [124]:
model2_sc_f=Sequential()
model2_sc_f.add(Dense(200, input_shape=(X_sc_train_fl.shape[1],), activation='relu'))
model2_sc_f.add(Dense(200, activation='relu'))
model2_sc_f.add(Dense(150, activation='relu'))
model2_sc_f.add(Dense(150, activation="relu"))
model2_sc_f.add(Dense(1, activation="relu"))
model2_sc_f.compile(loss='mse', optimizer='Adam')

In [152]:
model3_sc_f=Sequential()
model3_sc_f.add(Dense(200, input_shape=(X_sc_train_fl.shape[1],), activation='relu'))
model3_sc_f.add(Dropout(rate=0.3))
model3_sc_f.add(Dense(200, activation='relu'))
model3_sc_f.add(Dense(150, activation='relu'))
model3_sc_f.add(Dropout(rate=0.3))
model3_sc_f.add(Dense(150, activation="relu"))
model3_sc_f.add(Dense(1, activation="relu"))
model3_sc_f.compile(loss='mse', optimizer='Adam')

In [126]:
result2_sc_fl=wytrenuj_siec(model2_sc_f, X_sc_train_fl, X_sc_test_fl, y_sc_train_fl, y_sc_test_fl, 40)

Epoch 1/40
12967/12967 [==============================] - 4s 343us/step - loss: 142007855277.0621
Epoch 2/40
12967/12967 [==============================] - 2s 129us/step - loss: 36012136101.7969
Epoch 3/40
12967/12967 [==============================] - 2s 130us/step - loss: 33529774544.9735
Epoch 4/40
12967/12967 [==============================] - 2s 132us/step - loss: 32298795413.2725
Epoch 5/40
12967/12967 [==============================] - 2s 135us/step - loss: 31468249631.6668
Epoch 6/40
12967/12967 [==============================] - 2s 132us/step - loss: 30934845218.8849
Epoch 7/40
12967/12967 [==============================] - 2s 137us/step - loss: 30282852722.0520
Epoch 8/40
12967/12967 [==============================] - 2s 137us/step - loss: 29697373289.9773
Epoch 9/40
12967/12967 [==============================] - 2s 141us/step - loss: 29081435397.9425
Epoch 10/40
12967/12967 [==============================] - 2s 130us/step - loss: 28812609910.3163
Epoch 11/40
12967/12967 [===

In [127]:
print(result2_sc_fl)

163250.61309993168


In [153]:
result3_sc_fl=wytrenuj_siec(model3_sc_f, X_sc_train_fl, X_sc_test_fl, y_sc_train_fl, y_sc_test_fl, 55)
print(result3_sc_fl)

Epoch 1/55
12967/12967 [==============================] - 6s 443us/step - loss: 146181810393.4825
Epoch 2/55
12967/12967 [==============================] - 2s 162us/step - loss: 40279766638.0443
Epoch 3/55
12967/12967 [==============================] - 2s 167us/step - loss: 37899937555.2489
Epoch 4/55
12967/12967 [==============================] - 2s 162us/step - loss: 36415926974.0406
Epoch 5/55
12967/12967 [==============================] - 2s 153us/step - loss: 34996390993.4178
Epoch 6/55
12967/12967 [==============================] - 2s 154us/step - loss: 34929650986.6634
Epoch 7/55
12967/12967 [==============================] - 2s 153us/step - loss: 35517648524.8424
Epoch 8/55
12967/12967 [==============================] - 2s 154us/step - loss: 34897027123.2908
Epoch 9/55
12967/12967 [==============================] - 2s 154us/step - loss: 33649375065.2160
Epoch 10/55
12967/12967 [==============================] - 2s 155us/step - loss: 33849916353.0612
Epoch 11/55
12967/12967 [===

In [147]:
model4_sc_f=Sequential()
model4_sc_f.add(Dense(200, input_shape=(X_sc_train_fl.shape[1],), activation='relu'))
model4_sc_f.add(Dense(200, activation='relu'))
model4_sc_f.add(Dropout(rate=0.3))
model4_sc_f.add(Dense(200, activation='relu'))
model4_sc_f.add(Dense(150, activation='relu'))
model4_sc_f.add(Dropout(rate=0.3))
model4_sc_f.add(Dense(150, activation="relu"))
model4_sc_f.add(Dense(1, activation="relu"))
model4_sc_f.compile(loss='mse', optimizer='Adam')

In [148]:
result4_sc_fl=wytrenuj_siec(model4_sc_f, X_sc_train_fl, X_sc_test_fl, y_sc_train_fl, y_sc_test_fl, 50,10)
print(result4_sc_fl)

Epoch 1/50
12967/12967 [==============================] - 10s 771us/step - loss: 62532062596.9751
Epoch 2/50
12967/12967 [==============================] - 7s 513us/step - loss: 37106927494.4064
Epoch 3/50
12967/12967 [==============================] - 7s 516us/step - loss: 35165871327.4447
Epoch 4/50
12967/12967 [==============================] - 7s 543us/step - loss: 33001006526.7315
Epoch 5/50
12967/12967 [==============================] - 7s 542us/step - loss: 32490629738.1748
Epoch 6/50
12967/12967 [==============================] - 7s 548us/step - loss: 33378564764.6759
Epoch 7/50
12967/12967 [==============================] - 7s 536us/step - loss: 32637925561.9539
Epoch 8/50
12967/12967 [==============================] - 7s 545us/step - loss: 31222883283.1057
Epoch 9/50
12967/12967 [==============================] - 7s 525us/step - loss: 31045898936.3350
Epoch 10/50
12967/12967 [==============================] - 7s 521us/step - loss: 29217685091.8572
Epoch 11/50
12967/12967 [===

In [154]:
model3b_sc_f=Sequential()
model3b_sc_f.add(Dense(200, input_shape=(X_sc_train_fl.shape[1],), activation='relu'))
model3b_sc_f.add(Dropout(rate=0.3))
model3b_sc_f.add(Dense(200, activation='relu'))
model3b_sc_f.add(Dense(150, activation='relu'))
model3b_sc_f.add(Dropout(rate=0.3))
model3b_sc_f.add(Dense(150, activation="relu"))
model3b_sc_f.add(Dense(1, activation="relu"))
model3b_sc_f.compile(loss='mse', optimizer='Adam')

In [155]:
result3b_sc_fl=wytrenuj_siec(model3b_sc_f, X_sc_train_fl, X_sc_test_fl, y_sc_train_fl, y_sc_test_fl, 40,10)
print(result3b_sc_fl)

Epoch 1/40
12967/12967 [==============================] - 10s 771us/step - loss: 80977973795.6153
Epoch 2/40
12967/12967 [==============================] - 6s 489us/step - loss: 39705813180.1650
Epoch 3/40
12967/12967 [==============================] - 6s 489us/step - loss: 36549504959.3041
Epoch 4/40
12967/12967 [==============================] - 6s 498us/step - loss: 35205798654.4206
Epoch 5/40
12967/12967 [==============================] - 7s 562us/step - loss: 35130029103.3226
Epoch 6/40
12967/12967 [==============================] - 6s 493us/step - loss: 34154974094.0467
Epoch 7/40
12967/12967 [==============================] - 6s 479us/step - loss: 34664200641.7818
Epoch 8/40
12967/12967 [==============================] - 6s 471us/step - loss: 33991712828.0564
Epoch 9/40
12967/12967 [==============================] - 6s 466us/step - loss: 33491181992.3042
Epoch 10/40
12967/12967 [==============================] - 6s 500us/step - loss: 32719615348.8455
Epoch 11/40
12967/12967 [===